<a href="https://colab.research.google.com/github/sagareligar/Q-learning_CartPole_Problem/blob/master/Q_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# inspired by https://github.com/nikhilpodila/Reinforcement-Learning-Inverted-Pendulum

import numpy as np
import matplotlib.pyplot as plt
import time
import random 
t0=time.clock() # start timer for finding simulation time

# Mathematical model of the system
def cart_pole(force,theta,thetaDot,x,xDot):
  g=9.8
  M=1
  m=0.1
  total_mass = M+m
  length=0.5
  polemass_length=(m*length)
  tau=0.02
  fourthird=1.333
  temp = (force + polemass_length*thetaDot*thetaDot*np.sin(theta))/total_mass
  thetaacc = (g*np.sin(theta) - np.cos(theta)*temp)/(length*(fourthird - m*np.cos(theta)*np.cos(theta)/total_mass))
  xacc = temp - length *thetaacc*np.cos(theta)/total_mass

  xNext = x + tau*xDot
  xDotNext = xDot + tau*xacc
  thetaNext = theta + tau*thetaDot
  thetaDotNext = thetaDot + tau*thetaacc
  
  return [thetaNext,thetaDotNext,thetaacc,xNext,xDotNext]



In [0]:
#boxes

def getbox(theta,thetaDot,x,xDot):



  theta = np.rad2deg(theta)
  thetaDot = np.rad2deg(thetaDot)
  print(theta)
  print(thetaDot)
  if (x < -2.4 or x > 2.4  or theta<-12 or theta > 12):
    box=162
  else:

    if (theta<-6 and theta>=-12):
      thetaBucket = 0
    elif(theta<-1 and theta>=-6):
      thetaBucket = 1

    elif(theta<0 and theta>=-1):
      thetaBucket = 2

    elif(theta<1 and theta>=0):
      thetaBucket = 3

    elif(theta<6 and theta>=1):
      thetaBucket = 4

    elif(theta<12 and theta>=6):
      thetaBucket = 5



    if (x<-0.8 and x>=-2.4):
      xBucket = 0
    elif (x<=0.8 and x>=-0.8):
      xBucket = 1

    elif (x<=2.4 and x>=0.8):
      xBucket = 2



    if(xDot<-0.5):
      xDotBucket = 0
    elif(xDot>=-0.5 and xDot<=0.5):
      xDotBucket = 1
    else:
      xDotBucket = 2
      

    if(thetaDot<-50):
      thetaDotBucket = 0
    elif(thetaDot>=-50 and thetaDot<=50):
      thetaDotBucket = 1
    else:
      thetaDotBucket = 2

    box = 1+ np.ravel_multi_index((thetaBucket, thetaDotBucket,xBucket,xDotBucket), dims=(6,3, 3,3), order='F')
  return box
    





In [33]:
#Q-learning
NUM_BOXES = 163         # Number of states
ALPHA = 0.5             # Learning rate parameter
GAMMA = 0.999           # Discount factor for future reinforcements
Q = np.zeros((NUM_BOXES,2),dtype=float)  # State-Action Values
action = [10, -10]       # Action space (Force on cart)
MAX_FAILURES = 1000     # Maximum number of Failures allowed
MAX_STEPS = 150000      # Maximum number of steps (time) until Task success
epsilon = 0.5             # Epsilon value for epsilon-greedy strategy of action selection

# Initializing required parameters
steps = 0
failures = 0
thetaPlot = []
xPlot = []

#Pendulum state initialization
theta = 0
thetaDot = 0
x = 0
xDot = 0

# Get the index of the first state (Tabular method)
box = getbox(theta,thetaDot,x,xDot)

while(steps<=MAX_STEPS and failures<=MAX_FAILURES):


  steps = steps + 1

  if(np.random.random()>epsilon):
    #actionMax = max(Q[box,:])
    actionMax = np.argmax(Q[box,:])
    currentAction = action[actionMax]
  else:
    #currentAction = datasample(action,1)
    currentAction= random.sample(action, 1)[0]

  actionIndex = action.index(currentAction)  # index of chosen action
  [thetaNext,thetaDotNext,thetaacc,xNext,xDotNext] = cart_pole(currentAction,theta,thetaDot,x,xDot)
  
  thetaPlot.append((thetaNext*180)/(np.pi))
  xPlot.append(xNext)
  
  newBox = getbox(thetaNext,thetaDotNext,xNext,xDotNext)

  theta = thetaNext
  thetaDot = thetaDotNext
  x = xNext
  xDot = xDotNext

  if newBox == 162:
    r=-1
    Q[newBox,:] = 0
    # print(thetaPlot)
    # print("Hello")


    thetaPlot = [0]
    xPlot = [0]
    theta = 0
    thetaDot = 0
    x = 0
    xDot = 0

    newBox = getbox(theta,thetaDot,x,xDot)

    failures = failures + 1
    steps = 0

  else:
    r=0
 
  Q[box,actionIndex] = Q[box,actionIndex] +  ALPHA*(r + GAMMA*np.max(Q[newBox,:]) - Q[box,actionIndex])

  box=newBox

if(failures == MAX_FAILURES):
  print("Pole not balanced.")
else:
  print("Pole balanced successfully")
  print(thetaPlot)
  print("Theta")
  print(xPlot)
  print("xPlot")

  # plt.rcParams["figure.figsize"] = (15,10) # resizes figures for viewing
  # plt.figure()
  # plt.subplot(3,1,1)
  # plt.plot(failures+1,steps,'r',label='S per F')
  # plt.title('Number of steps taken to reach each Failure')
  # plt.legend()

  # plt.subplot(3,1,2)
  # length_thetatplot=range(15)
  # plt.plot(length_thetatplot,thetaPlot[0:15],'r',label='thetaplot')
  # plt.title("Pole Angle plot for the best trial")
  # plt.legend()

  # plt.subplot(3,1,3)
  # length_xplot=range(len(xPlot))
  # plt.plot(length_xplot,xPlot,'b',label='Xplot')
  # plt.title("Cart Position plot for the best trial")
  # plt.legend()


t1=time.clock()
total=t1-t0
print("Simulation time is {:.3f} seconds.\n" .format(total))

Streaming output truncated to the last 5000 lines.
0.5290947264982312
-30.46765608960869
-0.08025839529394257
-13.527720952022586
-0.3508128143343943
3.220934228638076
-0.2863941297616328
19.88385776100951
0.1112830254585574
3.0197671419032184
0.17167836829662175
19.82884774530662
0.5682553232027542
36.6569091469049
1.3013935061408521
20.062853009518008
1.7026505663312124
3.704247444646711
1.7767355152241464
-12.524033119792122
1.526254852828304
-28.728307285229196
0.9516887071237199
-11.480094202234278
0.7220868230790343
5.5915242529306735
0.8339173081376478
-10.953075878459057
0.6148557905684666
6.082007958879561
0.7364959497460578
23.048928600162697
1.1974745217493117
40.05358893176109
1.9985463003845334
57.200409591918245
3.1425544922228985
74.59034866556267
4.634361465534152
58.8285455720709
5.81093237697557
76.99289786986164
7.350790334372802
95.48234311736455
9.260437196720094
114.37981207104794
11.548033438141053
133.7552429998335
14.223138298137723
120.88878760122127
0.0
0.0
0